In [1]:
import os
import typing
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

key = "{AzureLanguageServicesKey}"
endpoint = "{AzureLanguageServicesEndpoint}"

text_analytics_client = TextAnalyticsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

In [2]:
documents = [
    """
    The room was exquisitely decorated, creating a welcoming ambiance. 
    However, the air conditioning was not functioning properly, which made our nights uncomfortably warm. 
    The breathtaking city view from our window partially made up for this inconvenience.
    """,
    """
    Our stay at the hotel was an absolute delight. 
    The room cleanliness was impeccable, and the staff were consistently friendly and attentive. 
    Waking up to the stunning oceanfront view each morning was a highlight of our trip.
    """,
    """
    The experience was far from pleasant due to the constant noise from nearby construction.
    Our room was not only stuffy but also felt outdated in terms of decor and amenities. 
    Additionally, the food offered at the hotel was mediocre, lacking both in taste and presentation.
    """,
    """
    The hotel provided a standard experience. 
    The rooms were clean and equipped with basic necessities, while the food was decent but unremarkable. 
    Overall, it was a forgettable stay, neither particularly good nor bad.
    """,
    """
    The service at the hotel was outstanding. 
    Our room was not only spacious but also exuded luxury, making our stay extremely comfortable. 
    The rooftop dining experience was a culinary delight, offering a wide variety of gourmet dishes that were both delicious and well-presented.
    """,
    """
    The hotel's central location was its best feature, making our city exploration quite convenient. 
    The room's modern aesthetics were appealing, yet the dining experience was underwhelming with limited food choices that lacked flavor.
    """,
    """
    We were disappointed with our cramped room, which barely had enough space to move around comfortably. 
    The view of a parking lot, marketed as a 'garden view', was a major letdown. Our overall experience was dampened by these aspects.
    """,
    """
    Our experience at the hotel was fairly average. 
    The room provided the basic comfort and cleanliness one would expect, but nothing stood out as particularly impressive. 
    The food served was satisfactory, though it lacked any unique or memorable qualities.
    """,
    """
    Our stay at the hotel was nothing short of fantastic. 
    The suite we were allocated was not only elegant but also offered a mesmerizing mountain view. 
    Dining at the hotel's restaurant was an exceptional experience, with each meal surpassing our expectations in both taste and presentation.
    """
]

In [3]:
result = text_analytics_client.analyze_sentiment(documents, show_opinion_mining=True)
doc_result = [doc for doc in result if not doc.is_error]

print("\nGeneral sentiment of each of these reviews")
positive_reviews = [doc for doc in doc_result if doc.sentiment == "positive"]
mixed_reviews = [doc for doc in doc_result if doc.sentiment == "mixed"]
negative_reviews = [doc for doc in doc_result if doc.sentiment == "negative"]
print("...We have {} positive reviews, {} mixed reviews, and {} negative reviews. ".format(
    len(positive_reviews), len(mixed_reviews), len(negative_reviews)
))


target_to_complaints: typing.Dict[str, typing.Any] = {}

for document in doc_result:
    for sentence in document.sentences:
        if sentence.mined_opinions:
            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target
                if target.sentiment == 'negative':
                    target_to_complaints.setdefault(target.text, [])
                    target_to_complaints[target.text].append(mined_opinion)



for target_name, complaints in target_to_complaints.items():
    print("Users have made {} complaint(s) about '{}', specifically saying that it's '{}'".format(
        len(complaints),
        target_name,
            "', '".join(
                [assessment.text for complaint in complaints for assessment in complaint.assessments]
            )
        ))


General sentiment of each of these reviews
...We have 2 positive reviews, 4 mixed reviews, and 3 negative reviews. 
Users have made 1 complaint(s) about 'nights', specifically saying that it's 'uncomfortably warm'
Users have made 1 complaint(s) about 'air conditioning', specifically saying that it's 'functioning properly'
Users have made 2 complaint(s) about 'room', specifically saying that it's 'stuffy', 'outdated', 'disappointed'
Users have made 1 complaint(s) about 'decor', specifically saying that it's 'outdated'
Users have made 1 complaint(s) about 'amenities', specifically saying that it's 'outdated'
Users have made 1 complaint(s) about 'food', specifically saying that it's 'mediocre'
Users have made 1 complaint(s) about 'taste', specifically saying that it's 'lacking'
Users have made 1 complaint(s) about 'presentation', specifically saying that it's 'lacking'
Users have made 1 complaint(s) about 'stay', specifically saying that it's 'forgettable', 'good'
Users have made 1 compl

In [4]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="{OPENAI_API_KEY}",
)


for doc in documents:

    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "assistant",
                "content": f"Analyse the text and if the review is negative show why the user complain about staing in the hotel in the short bullet points: '{doc}'"
            },
        ],
    )
    print(doc)
    print(completion.choices[0].message.content)
    print("***")


    The room was exquisitely decorated, creating a welcoming ambiance. 
    However, the air conditioning was not functioning properly, which made our nights uncomfortably warm. 
    The breathtaking city view from our window partially made up for this inconvenience.
    
- The air conditioning in the room was not working properly.
- This made the nights uncomfortably warm.
***

    Our stay at the hotel was an absolute delight. 
    The room cleanliness was impeccable, and the staff were consistently friendly and attentive. 
    Waking up to the stunning oceanfront view each morning was a highlight of our trip.
    
The review is positive, not negative. The user didn't complain about their stay at the hotel.
***

    The experience was far from pleasant due to the constant noise from nearby construction.
    Our room was not only stuffy but also felt outdated in terms of decor and amenities. 
    Additionally, the food offered at the hotel was mediocre, lacking both in taste and pres